In [26]:
import xml.etree.ElementTree as ET
import numpy as np
from scipy.spatial.transform import Rotation

def process_sites(root, site_dict=None, current_body=None):
    if site_dict is None:
        site_dict = {}  # Dictionary to store site information
    if current_body is None:
        current_body = None  

    for elem in root.iter():
        if elem.tag == 'body':
            current_body = elem.get('name')

        elif elem.tag == 'site':
            site_name = elem.get('name')
            site_pos = elem.get('pos', '0 0 0')

            if current_body is not None:
                if current_body not in site_dict:
                    site_dict[current_body] = []
                site_dict[current_body].append(np.fromstring(site_pos, sep=' '))

def process_geoms(root, geom_dict=None, current_body=None):
    if geom_dict is None:
        geom_dict = {}  # Dictionary to store geom information
    if current_body is None:
        current_body = None

    for elem in root.iter():
        if elem.tag == 'body':
            current_body = elem.get('name')

        elif elem.tag == 'geom':
            geom_name = elem.get('name')
            mesh_attr = elem.get('mesh')
            if not mesh_attr:
                pos_attr = elem.get('pos', '0 0 0')
                # euler_attr = elem.get('euler', '0 0 0')

                if current_body is not None:
                    if current_body not in geom_dict:
                        geom_dict[current_body] = []
                    # geom_dict[current_body].append(np.concatenate([np.fromstring(pos_attr, sep=' '), np.fromstring(euler_attr, sep=' ')]))
                    geom_dict[current_body].append(np.concatenate([np.fromstring(pos_attr, sep=' ')]))


def transform_site(original_pos, original_euler_degrees, site_pos):
    # Convert Euler angles to radians
    original_euler_radians = np.radians(original_euler_degrees)

    # Translation matrix
    translation_matrix = np.eye(4)
    translation_matrix[:3, 3] = original_pos

    # Rotation matrix
    rotation_matrix = np.eye(4)
    rotation_matrix[:3, :3] = Rotation.from_euler('zyx', original_euler_radians).as_matrix()

    # Combined transformation matrix
    combined_transform = np.dot(translation_matrix, rotation_matrix)

    # Transform the site position
    new_site_pos = np.dot(combined_transform, site_pos)
    translated_site_pos = new_site_pos[:3]

    return translated_site_pos


In [43]:
# Process A.xml for <site>
tree_a = ET.parse(r'D:\Mujoco\CyberMice\bones\forelimbs_240131.xml')
root_a = tree_a.getroot()

site_info_dict = {}
geom_info_dict = {}
process_sites(root_a, site_dict=site_info_dict)
process_geoms(root_a, geom_dict=geom_info_dict)

pelvis_site_positions = site_info_dict.get('RClavicle', [])
original_site_position = np.array(pelvis_site_positions)

carpi_geom_positions = geom_info_dict.get('RCarpi', [])
original_carpi_geom_position = np.array(carpi_geom_positions)

print(original_site_position)
# print(original_carpi_geom_position)

[[-0.00403751  0.0018592   0.00573398]]


In [44]:
# Original transformation parameters
original_pos = np.array([0.0065,-0.006,-0.001])
original_euler_degrees = np.array([25, 0, -20])

In [45]:
# target_position = np.zeros_like(original_carpi_geom_position)
# original_carpi_geom_position_add1 = np.hstack((original_carpi_geom_position, np.ones((original_carpi_geom_position.shape[0], 1))))
# for i in range(original_carpi_geom_position.shape[0]):
#     target_position[i] = transform_site(original_pos, original_euler_degrees, original_carpi_geom_position_add1[i])

# # Print the resulting matrix
# print(target_position)

[[0.0153762  0.00204188 0.00479925]
 [0.0153762  0.00204188 0.00479925]
 [0.01532735 0.00202074 0.00471304]
 [0.01532558 0.00202668 0.00486493]
 [0.01535798 0.00205006 0.00472415]
 [0.01531566 0.00199332 0.00485566]
 [0.015298   0.00199716 0.004788  ]
 [0.01540752 0.00200919 0.00491757]
 [0.0153762  0.00204188 0.00479925]
 [0.0154003  0.00200491 0.0047983 ]
 [0.0153762  0.00204188 0.00479925]
 [0.0153762  0.00204188 0.00479925]
 [0.01535683 0.00214594 0.0048281 ]
 [0.01540751 0.00197362 0.00477973]]


In [46]:
target_position = np.zeros_like(original_site_position)
original_site_position_add1 = np.hstack((original_site_position, np.ones((original_site_position.shape[0], 1))))
for i in range(original_site_position.shape[0]):
    target_position[i] = transform_site(original_pos, original_euler_degrees, original_site_position_add1[i])

# Print the resulting matrix
print(target_position)

[[ 0.00205504 -0.0040589   0.00439547]]


In [8]:
# L_Pelvis = original_site_position
# L_Pelvis[:,-1] = -1*L_Pelvis[:,-1]

# L_Pelvis_pos = np.array([0.003, 0.0015, 0.0015])
# L_Pelvis_euler_degrees = np.array([-5, 140, -5])

# L_Pelvis_target_position = np.zeros_like(L_Pelvis)
# L_Pelvis_add1 = np.hstack((L_Pelvis, np.ones((L_Pelvis.shape[0], 1))))
# for i in range(L_Pelvis.shape[0]):
#     L_Pelvis_target_position[i] = transform_site(L_Pelvis_pos, L_Pelvis_euler_degrees, L_Pelvis_add1[i])

# # Print the resulting matrix
# print(L_Pelvis_target_position)

[[-1.74690480e-02  9.41424358e-04 -4.14452075e-03]
 [-1.65346204e-02  3.86699778e-03 -8.67257693e-05]
 [-1.57157219e-02  3.27722523e-03 -1.18206013e-05]
 [-1.13462187e-02  2.95897147e-03  1.06590625e-04]
 [-1.21087596e-03  3.49011596e-03  3.15321047e-03]
 [-7.08952567e-03  2.65457841e-03  2.65455770e-03]
 [-1.77324580e-03  4.72580437e-03  2.12987557e-03]
 [-6.15438951e-03  2.96727078e-03  2.13615260e-03]
 [-1.84602043e-03  3.76580857e-03  9.31705331e-04]
 [-5.59946420e-03  2.84517108e-03  1.56121821e-03]
 [-1.73723008e-02  2.36728226e-03 -1.36571557e-03]
 [-1.76571686e-02  1.27831052e-03 -2.77408018e-03]
 [-1.49022162e-02  2.75403603e-03  2.93806227e-03]
 [-1.75225711e-02  8.88558345e-04 -2.88577953e-03]
 [-1.47968810e-02  6.55230007e-04 -3.01060722e-04]
 [-1.43934816e-02  7.30675671e-04 -2.65326867e-04]
 [-1.42781798e-02  5.60075702e-04 -1.46158319e-04]
 [-1.70785883e-02  2.12194741e-03 -1.48252234e-03]
 [-1.88779132e-02 -7.34804888e-04 -4.31926197e-03]
 [-1.69404757e-02  4.47613371e-